# Final Project Phase 3 (Webscraping)

### Alicia Chen (ac2596), Stephanie Chow (sac342), Michelle Wang (mqw4)

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [2]:
# Dataset 1
# Source: kpop_vids_yt.htm

with open("kpop_mvs_1000_entries.htm", encoding = "utf-8") as file_reader:
    soup = BeautifulSoup(file_reader, "lxml")
    
for tr in soup.find_all("tr", class_ = "clickable"):
    td_list = tr.find("td").get_text()

print(td_list)

1000. Flash(dance)


In [3]:
def period_to_comma(old_array):
    new_array = []
    [new_array.append(i.replace(".", ",")) for i in old_array]
    return new_array

def comma_to_period(old_array):
    new_array = []
    [new_array.append(i.replace(",", ".")) for i in old_array]
    return new_array

def no_comma(data):
    return re.sub(",", "", data)

In [4]:
data = []
typeofartist = []
song_raw = []
artist = []

for tr in soup.find_all("tr", class_ = "clickable"):
    [data.append(td.get_text()) for td in tr.find_all("td")]
    typeofartist.append(tr.div['title'])

# indexing each value position
song_index, artist_index = 0, 1

while artist_index < len(data):
    song_raw.append(data[song_index])
    artist.append(re.sub(" ", "", data[artist_index]))
    song_index += 9
    artist_index += 9
    
arrlikes_raw=[]
arrviews_raw=[]
arrdays_raw = []

for views, likes, days in zip(soup.find_all("td", class_='c0'), soup.find_all("td", class_='c3'), soup.find_all("td", class_='c6')):
    arrlikes_raw.append(likes.get_text()[:])
    arrviews_raw.append(views.get_text()[:])
    arrdays_raw.append(days.get_text()[:])
    
arrlikes = period_to_comma(arrlikes_raw)
arrviews = period_to_comma(arrviews_raw)
arrdays = period_to_comma(arrdays_raw)

# cleaning song name data without numbers
songs = []
for song in song_raw:
    replace_num = re.sub('\d+. ', '', song)
    songs.append(replace_num.strip())

# cleaning likes data and splitting into two parts
likes_total_raw = []
likes_ratio_raw = []

for likes in arrlikes:
    likes_total_raw.append(re.sub(' \(\d+,\d+%\)', "", likes))
    ratio = re.findall('(\d+,\d+%)', likes)
    likes_ratio_raw.append(ratio[0])

likes_total = period_to_comma(likes_total_raw)
likes_ratio = comma_to_period(likes_ratio_raw)

days = []
for day in arrdays:
    new_days = re.sub(" days", "", day)
    days.append(no_comma(new_days))
    
arryears = np.array(days, dtype="float")
arryears = arryears/365

In [5]:
df = pd.DataFrame({"Song Title": songs, 
                   "Artist": artist, 
                   "Type of Artist": typeofartist, 
                   "MV Views": arrviews, 
                   "Likes": likes_total, 
                   "Likes Ratio": likes_ratio,
                   "Days Old" : arrdays_raw,
                   "Years Old" : arryears
                  })
categories = ["Song Title", "Artist", "Type of Artist", "Likes", "Likes Ratio", "MV Views", "Days Old", "Years Old"]
df = df.reindex(columns = categories)
df.head()

,Song Title,Artist,Type of Artist,Likes,Likes Ratio,MV Views,Days Old,Years Old
0,Gangnam Style,PSY,male soloist,"17,848,966",88.2%,"3,612,465,662",2.859 days,7.832877
1,Boy with luv,BTS,male group,"14,942,987",94.3%,"776,602,125",397 days,1.087671
2,DNA,BTS,male group,"13,257,145",94.9%,"978,195,463",968 days,2.652055
3,Kill this Love,Blackpink,female group,"12,920,236",93.9%,"826,027,265",405 days,1.109589
4,Ddu-du Ddu-du,Blackpink,female group,"12,544,634",94.5%,"1,144,730,047",698 days,1.912329


In [6]:
# function to strip the numbers off from the beginning of the name/ artist
# EX: 1. BTS --> BTS
def strip_numbers(data, lst):
    counter = 0
    while counter < len(data):
        replace_num = re.sub('\d+. ', '', data[counter])
        no_space = re.sub(' ', '', replace_num)
        lst.append(no_space)
        counter += 9

In [7]:
# Dataset 2

with open("kpop_artists_300_entries.htm", encoding = "utf-8") as file_reader:
    soup2 = BeautifulSoup(file_reader, "lxml")
    
data = []
typeofartist = []
artist = []

for tr in soup2.find_all("tr", class_ = "clickable"):
    [data.append(td.get_text()) for td in tr.find_all("td")]
    typeofartist.append(tr.div['title'])

# get rid of the numbers in front of the names
strip_numbers(data, artist)

arrlikes_raw=[]
arrsales_raw=[]
arrviews_raw=[]

for views, likes, sales in zip(soup2.find_all("td", class_='c0'), soup2.find_all("td", class_='c7'), soup2.find_all("td", class_='c8')):
    arrlikes_raw.append(likes.get_text()[:])
    arrsales_raw.append(sales.get_text()[:-3])
    arrviews_raw.append(views.get_text()[:])

arrlikes = period_to_comma(arrlikes_raw)
arrsales = period_to_comma(arrsales_raw)
arrviews = period_to_comma(arrviews_raw)

In [8]:
# dataframe that includes the Korean name of the art
need = pd.DataFrame({"Artist": artist, 
                    "Type of Artist": typeofartist, 
                    "Total Views": arrviews, 
                    "Likes/MV": arrlikes, 
                    "Sales": arrsales})
categories = [ "Artist", "Type of Artist", "Likes/MV", "Sales", "Total Views"]
need = need.reindex(columns = categories)
need.head()

,Artist,Type of Artist,Likes/MV,Sales,Total Views
0,BTS(방탄소년단),male group,"2,400,603","21,047,066","11,914,564,663"
1,Blackpink(블랙핑크),female group,"3,495,869","652,585","6,236,964,281"
2,Twice(트와이스),female group,"951,944","4,572,592","5,119,889,077"
3,EXO(엑소),male group,"1,285,821","10,273,845","2,690,343,428"
4,GOT7(갓세븐),male group,"534,887","3,386,454","2,077,803,857"


In [9]:
# data without the Korean names to merge
new_data = []
for tr in soup2.find_all("tr", class_ = "clickable"):  
    row = tr.find('td')
    small = row.small
    if type(small) != type(None):
        small.decompose()
    [new_data.append(td.get_text()) for td in tr.find_all("td")]

artists = []
strip_numbers(new_data, artists)

new = need.copy()
new['Artist'] = artists
new.head()

,Artist,Type of Artist,Likes/MV,Sales,Total Views
0,BTS,male group,"2,400,603","21,047,066","11,914,564,663"
1,Blackpink,female group,"3,495,869","652,585","6,236,964,281"
2,Twice,female group,"951,944","4,572,592","5,119,889,077"
3,EXO,male group,"1,285,821","10,273,845","2,690,343,428"
4,GOT7,male group,"534,887","3,386,454","2,077,803,857"


In [10]:
combined_data = pd.merge(df, new)
combined_data.head()

,Song Title,Artist,Type of Artist,Likes,Likes Ratio,MV Views,Days Old,Years Old,Likes/MV,Sales,Total Views
0,Gangnam Style,PSY,male soloist,"17,848,966",88.2%,"3,612,465,662",2.859 days,7.832877,"1,997,592","518,974","6,054,464,700"
1,Gentleman,PSY,male soloist,"6,007,998",87.3%,"1,312,037,506",2.587 days,7.087671,"1,997,592","518,974","6,054,464,700"
2,Daddy,PSY,male soloist,"3,799,519",93.8%,"493,121,830",1.626 days,4.454795,"1,997,592","518,974","6,054,464,700"
3,New Face,PSY,male soloist,"1,581,019",95.0%,"205,665,596",1.099 days,3.010959,"1,997,592","518,974","6,054,464,700"
4,I Luv it,PSY,male soloist,"973,795",94.7%,"101,822,612",1.099 days,3.010959,"1,997,592","518,974","6,054,464,700"


In [11]:
combined_data.to_csv('combined.csv', encoding='utf-8')